**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов (NLTK model)

In [1]:
import gzip
import requests
from bs4 import BeautifulSoup

In [2]:
# url='http://lib.ru/NEWPROZA/LOBAS/taxisty.txt'
# text = BeautifulSoup(requests.get(url).text).get_text()
# with gzip.open('taxisty.txt.gz','wt') as f: f.write(text)

# # with gzip.open('taxisty.txt.gz','rt') as f: text = f.read()

# text = text[1030:-7261].strip() # выкидываем заголовок и хвост страницы 
# print(f'символов:{len(text)}\n---------------\n'%())
# print(text[:343])

In [3]:
url='http://az.lib.ru/d/dostoewskij_f_m/text_0080.shtml'
text = BeautifulSoup(requests.get(url).text).get_text()
with gzip.open('dostoewskij.txt.gz','wt') as f: f.write(text)

# with gzip.open('dostoewskij.txt.gz','rt') as f: text = f.read()

text = text[2876:-664184].strip() # выкидываем заголовок и хвост страницы 
print(f'символов:{len(text)}\n---------------\n'%())
print(text[:355])

символов:1279540
---------------

Приступая к описанию недавних и столь странных событий, происшедших в нашем, доселе ничем не отличавшемся городе, я принужден, по неумению моему, начать несколько издалека, а именно некоторыми биографическими подробностями о талантливом и многочтимом Степане Трофимовиче Верховенском. Пусть эти подробности послужат лишь введением к предлагаемой хронике, 


In [4]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.8.1


In [5]:
from nltk.tokenize import sent_tokenize as nltk_sentence_split
nltk_sentence_split?

In [6]:
from tqdm.auto import tqdm
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

sentences = [ 
    nltk_tokenize_word(s,language='russian') # разбиваем предложения на слова
    for s in tqdm(nltk_sentence_split(text,language='russian')) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(sentences)))
display( sample(sentences,1) )

  0%|          | 0/14424 [00:00<?, ?it/s]

предложений: 14424



[['Я',
  'выйду',
  'на',
  'крыльцо',
  ',',
  'и',
  'предположите',
  ',',
  'что',
  'я',
  'ничего',
  'не',
  'понимаю',
  'и',
  'что',
  'я',
  'безмерно',
  'ниже',
  'вас',
  'человек',
  '.']]

In [7]:
sentences = sentences[:1024*7] # ограничиваем датасет для ускорения процеса 

In [8]:
%%time

from nltk.lm.preprocessing import padded_everygram_pipeline 

ngram_len = 2

# генерируем учебный датасет
train, vocab = padded_everygram_pipeline(ngram_len, sentences)

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 13.1 µs


In [9]:
# собираем модель

# from nltk.lm import MLE as LangModel 
from nltk.lm import Laplace as LangModel

model = LangModel(ngram_len) 
model.fit(train, vocab)

display(len(model.vocab))

20961

In [10]:
# генерируем продолжения
for sentence in sample(sentences,30): # выбираем рандомно 10 предложений
    if len(sentence)<10: continue
    # берём начало предложения
    sentence_ = sentence[:-(len(sentence)//4)]
    # генерируем возможные продолжения
    result = model.generate(3, text_seed=sentence_) 
    print(  ' '.join(sentence_)  + ' ... ' + str( result ) + '\n' )

Объявлялось лицо новое , в котором все ошиблись , лицо почти с ... ['извинениями', ',', 'хотя']

Но тут подвернутся Петр Степанович , и стало происходить ... ['нечто', 'нескладное', ',']

Но послушай , однако , надо , чтобы не ... ['могла', '...', 'давно']

А за вами она спокойна , и я ... ['долго', 'спустя', 'припомнили']

Впрочем , я всегда сознавал себя русским ... да настоящий русский и не может быть иначе , ... ['в', 'то', 'сходиться']

- так и вскрикнул Степан Трофимович , -- точно его самого вдруг ... ['вдовица', '.', '</s>']

Скажите мне : ваш-то заяц пойман ли аль еще ... ['к', 'столу', '.']

- Люблю , коли с обществом , кла-сси-чес ... значит , о-бразо-о-ваннейший ... отставной капитан Игнат Лебядкин , к услугам мира и друзей ... ... ['вовсе', 'нет', ',']

- Пойдемте , -- сказал он , -- я ... ['завещать', 'мой', 'вопросительный']

У Липутина много фантазии , вместо пустяков горы ... ['далеко', 'не', 'откажетесь']

Один заметил , что это наилучший исход и что умнее мальчик и н

In [11]:
# from nltk.util import bigrams
from nltk.util import ngrams

# оцениваем насколько хорошо модель предсказывает слова из датасета
text_ngrams = [ ng for s in sentences for ng in ngrams(s,ngram_len) ]

print( 'perplexity:', model.perplexity( text_ngrams ) )

perplexity: 4032.6262712969137


In [12]:
display( text_ngrams[:3] )

[('Приступая', 'к'), ('к', 'описанию'), ('описанию', 'недавних')]